#### GROMACS for production <font color="DarkSeaGreen">/ GROMACS-on-Colab</font> [<img src="https://github.githubassets.com/favicons/favicon.png" width="16">](https://github.com/bioinfkaustin/gromacs-on-colab)
<small>Suite: [`Build_to_Google_Drive.ipynb`](https://colab.research.google.com/github/bioinfkaustin/gromacs-on-colab/blob/main/Build_to_Google_Drive.ipynb) | [`GROMACS_for_CHARMM-GUI.ipynb`](https://colab.research.google.com/github/bioinfkaustin/gromacs-on-colab/blob/main/GROMACS_for_CHARMM-GUI.ipynb) | `GROMACS_for_production.ipynb` | [`Trajectory_analysis_tools.ipynb`](https://colab.research.google.com/github/bioinfkaustin/gromacs-on-colab/blob/main/Trajectory_analysis_tools.ipynb)</small>

#### Documentation
Please click ***cells hidden*** below to show the documentation for this notebook.
##### License
> This notebook as a work of software is licensed under the terms of the [AGPL-3.0](https://opensource.org/licenses/AGPL-3.0) or later.
##### About this software
> This notebook runs or extends a **GROMACS production simulation**.
>
> It operates within a **project folder** containing the initial condition `conf.gro`, the topology `topol.top` (and probably `toppar/`), and the simulation parameters file `grompp.mdp` alongside its prerequisites (the group labels `index.ndx` and so on).
>
> All inputs must have the [GROMACS default names](https://manual.gromacs.org/current/onlinehelp/gmx-grompp.html) (e.g. if you need to provide reference coordinates for restraints, a file called `restraint.gro` must be present).
##### Installation
> The installation notebook, [`Build_to_Google_Drive.ipynb`](https://colab.research.google.com/github/bioinfkaustin/gromacs-on-colab/blob/main/Build_to_Google_Drive.ipynb), must be run before using this notebook.

In [ ]:
import os
import re
import shutil

#@markdown Specify the location of the **GROMACS project folder** to simulate. It should contain `conf.gro` and `topol.top` (and probably `toppar/`), the simulation parameters file `grompp.mdp`, and any optional input files required by those parameters (such as `index.ndx` or `restraint.gro`).
project_folder = "{GoogleDrive}/GROMACS/7FBF_FABPH_vs_octanoic_acid" #@param {type: "string"}
# default: {GoogleDrive}/GROMACS/7FBF_FABPH_vs_octanoic_acid

#@markdown Choose for how long the simulation should run.
simulation_duration_ns = 10 #@param {type: "integer"}
# default: 10

#@markdown Provide a unique filename prefix for this simulation.
output_prefix = "sim" #@param {type: "string"}
# default: sim

#@markdown If applicable, please see also the advanced settings below. **After filling in this form, run the notebook by clicking *Runtime -> Run all* in the toolbar.**

#@markdown \
#@markdown **Early stopping**
#@markdown
#@markdown Optionally, a group may be specified for which the RMSD from the initial conformation should be monitored. The run stops if a threshold RMSD (Angstroms) is exceeded for 0.5 ns.
rmsd_group = "LIG" #@param {type: "string"}
rmsd_early_stop_threshold_A = 12.0 #@param {type: "number"}
# default: 12.0

# Google Drive
if not os.path.isdir("/content/drive/MyDrive"):
  from google.colab import drive
  drive.mount("/content/drive")
if not os.path.isdir("/content/drive/MyDrive"):
  raise RuntimeError("Error: could not connect to Google Drive")

# Validate the input values
def _path(s, exists=False):
  if "{GoogleDrive}" in s and not s.startswith("{GoogleDrive}"): raise ValueError(f"Error: {{GoogleDrive}} is a path prefix, but appears later: {s}")
  s = s.format(GoogleDrive="/content/drive/MyDrive")
  if exists and not os.path.isdir(s): raise FileNotFoundError(f"Error: folder not found: {s}")
  return os.path.abspath(s)

def parse(s, mandatory=False, path=False, exists=False):
  s = s.strip()
  if mandatory and not s:
    raise ValueError("Error: mandatory setting without value")
  if path:
    s = _path(s, exists=exists)
  return s

project_folder = parse(project_folder, mandatory=True, path=True, exists=True)

if simulation_duration_ns <= 0:
  raise RuntimeError(f"Error: simulation duration must be more than 0 ns, but got: {simulation_duration_ns} ns")
simulation_duration_ps = 1000 * simulation_duration_ns

output_prefix = output_prefix.strip()
if not output_prefix:
  raise RuntimeError("Error: an output prefix must be provided")
if not re.match(r"^[0-9a-zA-Z]+$", output_prefix):
  raise RuntimeError(f"Error: special characters are not allowed in output prefix, but got: {output_prefix}")
if output_prefix in ("dat", "pre", "lig"):
  raise RuntimeError(f"Error: reserved output prefix: {output_prefix}")

rmsd_group = rmsd_group.strip()

if rmsd_early_stop_threshold_A is None:
  rmsd_early_stop_threshold_A = 0.0
rmsd_early_stop_threshold_nm = rmsd_early_stop_threshold_A / 10.

# Make sure that the notebook is in the start folder
if "START" not in os.environ or not os.environ["START"]:
  %env START={os.getcwd()}
else:
  %cd {os.environ["START"]}

# Use a clean scratch directory for the rest of the run
try:
  shutil.rmtree("scratch")
except FileNotFoundError:
  pass
os.makedirs("scratch")
%cd "scratch"

In [ ]:
%%bash -s "$project_folder" "$output_prefix"
project_folder="$1"
output_prefix="$2"

#@markdown Extract the system from the project folder.
if [[ ! -z "$(ls -A)" ]]; then
  exit 0 # already extracted
fi
if [[ ! -d "${project_folder}" ]]; then
  echo "Error: folder not found: ${project_folder}" >&2
  exit 1
fi

pushd "${project_folder}"
cp "grompp.mdp" "conf.gro" "restraint.gro" "index.ndx" "topol.top" "${START}/scratch/" 2> /dev/null

top_level_dir="$(realpath .)"
function get_includes_recursively {
  f="$1"
  sed -E "/^#include/!d; s/^#include +['\"]//; s/['\"]$//" "${f}" | while read -r g; do
    d="$(dirname "${g}")"
    b="$(basename "${g}")"
    pushd "${d}" > /dev/null
    get_includes_recursively "${b}"
    echo "$(realpath --relative-to="${top_level_dir}" "${b}")"
    popd > /dev/null
  done
}
get_includes_recursively "topol.top" | while read -r f; do
  cp --parents "${f}" "${START}/scratch/"
done

i=1
part="$(printf "part%04i" "$i")"
while [[ -s "${output_prefix}.${part}.tar.gz" ]]; do
  if gunzip --test "${output_prefix}.${part}.tar.gz"; then
    tar -xzf "${output_prefix}.${part}.tar.gz" -C "${START}/scratch"
  else
    echo "Warning: corrupted part file: ${output_prefix}.${part}.tar.gz" >&2
    break
  fi
  i=$(($i + 1))
  part="$(printf "part%04i" "$i")"
done
popd

if [[ ! -s "grompp.mdp" || ! -s "conf.gro" || ! -s "topol.top" ]]; then
  echo "Error: essential files not found: grompp.mdp, conf.gro, topol.top" >&2
  exit 1
fi

In [ ]:
#@markdown In the following cells, applications are downloaded from a **persistent cache** in your Google Drive.
#@markdown This cell sets up the cache folder.
storage = "/content/drive/MyDrive/gromacs-on-colab"
%env STORAGE={storage}

In [ ]:
%%bash
#@markdown **GROMACS** is installed from cache.
if [[ -d "/usr/local/gromacs" ]]; then
  exit 0 # already installed
fi
gromacs_vers="2023.2" #@param {type: "string"}
cache_gromacs="${STORAGE}/gromacs-${gromacs_vers}.tar.gz"
if [[ -s "${cache_gromacs}" ]]; then
  tar -xzf "${cache_gromacs}" -C "/usr/local"
else
  echo "Error: GROMACS installation not found" >&2
  echo "(Have you installed GROMACS to your Google Drive?)" >&2
  exit 1
fi

In [ ]:
%%writefile "run.bash"
project_folder="$1"
simulation_duration_ps=$2
rmsd_group="$3"
rmsd_early_stop_threshold_nm=$4
prefix="$5"

#@markdown Create a script to run the production simulation.
source "/usr/local/gromacs/bin/GMXRC.bash"
export GMX_MAXCONSTRWARN=-1
export GMX_CUDA_GRAPH=1 # experimental upgrade

{
  echo "***"
  echo "${project_folder}"
  date "+%F %T"
  echo "---"
  echo "$(nproc) cores, $(free -m | awk 'NR == 2 { print $2 }') MiB"
  nvidia_smi="$(nvidia-smi --query-gpu="name,memory.total" --format="csv,noheader")"
  if (( $? == 0 )); then
    echo "$nvidia_smi"
  fi
  echo "***"
  echo ""
} | tee -a "${prefix}.summary"

# Get the runtime of each individual run
sim_dt=$(awk '$1 == "dt" { print $3 }' "grompp.mdp")
sim_nsteps=$(awk '$1 == "nsteps" { print $3 }' "grompp.mdp")
block_ps=$(perl -e "printf(\"%.0f\n\", ${sim_dt} * ${sim_nsteps})")

# Is this the first ever run?
if [[ -s "${prefix}.cpt" ]]; then
  skip_because_continuation=true
else
  skip_because_continuation=false
  cmd=( gmx grompp -f "grompp.mdp" -o "${prefix}.tpr" -c "conf.gro" -p "topol.top" -maxwarn 999 )
  if [[ -s "restraint.gro" ]]; then
    cmd+=( -r "restraint.gro" )
  fi
  if [[ -s "index.ndx" ]]; then
    cmd+=( -n "index.ndx" )
  fi
  "${cmd[@]}"

  {
    echo "Block: 0 ps to ${block_ps} ps / ${simulation_duration_ps} ps"
    date "+%F %T"
    echo ""
  } | tee -a "${prefix}.summary"
  gmx mdrun -v -stepout 1000 -deffnm "${prefix}" -noappend -bonded gpu || exit $?
fi

[[ -s "${prefix}.part0001.trr" ]] && trr_frames="$(gmx check -f "${prefix}.part0001.trr" 2>&1 | awk '$1 ~ /^Coords/ { print $2 }')" || trr_frames=0
[[ -s "${prefix}.part0001.xtc" ]] && xtc_frames="$(gmx check -f "${prefix}.part0001.xtc" 2>&1 | awk '$1 ~ /^Coords/ { print $2 }')" || xtc_frames=0
if (( $trr_frames == 0 && $xtc_frames == 0 )); then
  echo "Error: no trajectory found: ${prefix}.part0001.xtc or ${prefix}.part0001.trr" >&2
  exit 0
elif (( $trr_frames >= $xtc_frames )); then
  xtcext="trr"
  ndecarg="-ndec 4"
else
  xtcext="xtc"
  ndecarg=""
fi

while true; do
  latest_part="$(ls | egrep '^'"${prefix}"'\.part[0-9]+\.log$' | sort -V | tail -n1 | egrep -o 'part[0-9]+')"
  if $skip_because_continuation; then
    skip_because_continuation=false
  else
    {
      grep "^Performance: " -B3 -A1 "${prefix}.${latest_part}.log"
      echo ""
    } | tee -a "${prefix}.summary"
    tar -vczf "${prefix}.${latest_part}.tar.gz" "${prefix}.summary" "${prefix}.tpr" "${prefix}.cpt" "${prefix}.${latest_part}.*"
    cp "${prefix}.${latest_part}.tar.gz" "${project_folder}/"
  fi

  gmx trjconv -f "${prefix}.${latest_part}.${xtcext}" -s "${prefix}.tpr" -dump 999999999999 -o "last.gro" <<< "0"
  t=$(($(head -n1 "last.gro" | egrep -o 't= [0-9]+' | sed 's/^t= //')))
  if [[ -z "$t" ]] || (( $t == 0 )); then
    echo "Error: could not read time from trajectory" >&2
    exit 1
  fi
  rm "last.gro"

  if (( $t >= $simulation_duration_ps )); then
    {
      echo "Detected: current simulation time ${t} ps exceeded target time ${simulation_duration_ps} ps"
      echo ""
    } | tee -a "${prefix}.summary"
    break
  fi

  if [[ ! -z "${rmsd_group}" && ! -z "${rmsd_early_stop_threshold_nm}" ]] && perl -e "${rmsd_early_stop_threshold_nm} > 0.001 ? exit 0 : exit 1"; then
    gmx trjconv -s "${prefix}.tpr" -f "${prefix}.${latest_part}.${xtcext}" -pbc nojump -o "nojump.xtc" <<< 0
    gmx rms -s "${prefix}.tpr" -f "nojump.xtc" -b $(($t - $block_ps)) < <(echo "C-alpha"; echo "${rmsd_group}")
    rmsd_steps=$(sed '/^[#@&]/d; /^ *$/d' "rmsd.xvg" | wc -l)
    rmsd_alarm=$(sed '/^[#@&]/d; /^ *$/d' "rmsd.xvg" | awk "\$2 > ${rmsd_early_stop_threshold_nm}" | wc -l)
    rm "nojump.xtc"
    if perl -e "$rmsd_alarm > $rmsd_steps / 2 ? exit 0 : exit 1"; then
      {
        echo "Detected: RMSD of ${rmsd_group} exceeded threshold ${rmsd_early_stop_threshold_nm} nm"
        echo ""
      } | tee -a "${prefix}.summary"
      break
    fi
  fi

  {
    echo "Block: ${t} ps to $((${t} + ${block_ps})) ps / ${simulation_duration_ps} ps"
    date "+%F %T"
    echo ""
  } | tee -a "${prefix}.summary"
  gmx convert-tpr -s "${prefix}.tpr" -extend $block_ps -o "tprout.tpr"
  mv "tprout.tpr" "${prefix}.tpr"
  gmx mdrun -cpi "${prefix}.cpt" -v -stepout 1000 -deffnm "${prefix}" -noappend -bonded gpu || exit $?
done

{
  echo "***"
  echo "End"
  date "+%F %T"
  echo "***"
  echo ""
} | tee -a "${prefix}.summary"
cp "${prefix}.summary" "${project_folder}/${prefix}_reference.summary"

gmx trjcat -f "${prefix}.part"*".${xtcext}" -o "${prefix}.${xtcext}"
gmx trjconv -f "${prefix}.${xtcext}" -s "${prefix}.tpr" -pbc nojump $ndecarg -o "nojump.xtc" <<< "0"
gmx trjconv -f "nojump.xtc" -s "${prefix}.tpr" -fit progressive $ndecarg -o "progressive.xtc" < <(echo "C-alpha"; echo "0")
gmx trjconv -f "progressive.xtc" -s "${prefix}.tpr" -pbc mol $ndecarg -o "${prefix}_reference.xtc" <<< "0"
cp "${prefix}_reference.xtc" "${project_folder}/"
rm "nojump.xtc" "progressive.xtc"

exit 0

In [ ]:
#@markdown Execute the simulation script.
#@markdown Run a loop of blocks (typically 1 ns) until the **production simulation** is complete -- each loop iteration saves a partial output.
!bash "run.bash" "$project_folder" "$simulation_duration_ps" "$rmsd_group" "$rmsd_early_stop_threshold_nm" "$output_prefix"
!sleep 10

In [ ]:
#@markdown Finally, disconnect the runtime. (This option is ignored if the project folder is not in your Google Drive.)
disconnect = True #@param {type: "boolean"}
if disconnect and project_folder.startswith("/content/drive/MyDrive/"):
  from google.colab import drive, runtime
  drive.flush_and_unmount()
  runtime.unassign()